In [3]:
from dataclasses import dataclass
import hl7apy
from hl7apy.core import Message
import json
import pandas as pd

In [4]:
import sys
sys.executable

'C:\\Users\\maxst\\AppData\\Local\\Programs\\Python\\Python310\\python.exe'

In [16]:
df = pd.read_csv('tumorboard_testfall.csv')

In [17]:
df

,Patientname,Geburtsdatum,Adresse des Patienten,Orbis PID,Krankheits ID,Datum Vorstellung Tumorboard,Chirurgischer Status,Tumorentität,angemeldet von,Tel.-Nr. vom Anmelder,...,Vorherige Tumorboards,Letzte Empfehlung umgesetzt?,Für ASV geeignet?,Diskussionsgrundlage,Tumorboardempfehlung,Unterschriftfeld,Name der Untershreiber,Funktionsbezeichnung,Anmekrung nach Vidierung,Eingabefeld für Anmerkung nach Vidierung
0,Maximilian Schmidt,1965-08-17,"Schillerstraße 12, 76133 Karlsruhe",12345678,2025-GI-0001,2025-05-23,Postoperativ R0-Resektion,Adenokarzinom des Sigma,Dr. med. Anna Becker,0721 974-2255,...,Keine,NaN,Ja,Postoperative Stadienzuordnung; adjuvante Ther...,6 Zyklen adjuvantes FOLFOX6; begleitende Ernäh...,placeholder,Prof. Dr. med. Rüdiger,Leitender Oberarzt Viszeralchirurgie,Keine zusätzlichen Hinweise,-


#### Hole Liste von Patienten aus LIS

In [6]:
patient_list = "GI-Tumorboard-Liste.json"
with open(patient_list, "r", encoding="utf-8") as f:
    date_list = json.load(f)

In [7]:
dates = list(date_list.keys())
dates

['25.05.2025']

In [8]:
patient_list = date_list[dates[0]]
patient_list[0]

{'Name': 'Max', 'Nachname': 'Mustermann', 'PID': '12345', 'TumorID': '024901'}

In [20]:
@dataclass
class FallTumorBoard:
    vorname: str
    nachname: str
    pid: int
    tumorid: int
    tumorentität: str = None
    histopathologie: str = None
    primaertumor: str = None
    nodalstatus: str = None
    entscheider: str = None

In [21]:
def json_to_dataclass(fall_dict):
    fall_instance = FallTumorBoard(
        vorname=fall_dict['Name'],
        nachname=fall_dict['Nachname'],
        pid=fall_dict['PID'],
        tumorid=fall_dict['TumorID']
    )
    return fall_instance

In [22]:
fall_1 = json_to_dataclass(patient_list[0])

In [23]:
fall_1.tumorid

'024901'

#### Hole relevante Daten zum Tumorfall

In [24]:
def get_patho_info(tumorid):
    """
    Jede Tumor Board Fall hat eine Tumormeldung.
    Diese Tumormeldung hat Parametern, die man aufrufen kann.
    """
    with open("pw_tumormeldungen.json", mode="r", encoding="utf-8") as f:
        tumormeldungen = json.load(f)

    return tumormeldungen[tumorid]

In [25]:
patho_info = get_patho_info(fall_1.tumorid)
patho_info

{'tumorentität': 'anus',
 'histopathologie': 'RAS-Mutation',
 'primaertumor': 'pTis',
 'nodalstatus': 'platzhalter',
 'entscheider': 'mustermann@krankenhaus.de'}

In [27]:
fall_1.tumorentität = patho_info['tumorentität']
fall_1.histopathologie = patho_info['histopathologie']
fall_1.primaertumor = patho_info['primaertumor']
fall_1.nodalstatus = patho_info['nodalstatus']
fall_1.entscheider = patho_info['entscheider']

In [28]:
fall_1

FallTumorBoard(vorname='Max', nachname='Mustermann', pid='12345', tumorid='024901', tumorentität='anus', histopathologie='RAS-Mutation', primaertumor='pTis', nodalstatus='platzhalter', entscheider='mustermann@krankenhaus.de')

In [ ]:
def get_notes_from_previos_tb(tumor_id):
    """Notes are always carried forward""".

#### Create ADT

In [18]:
last_name = 'Mustermann'
first_name = 'Max'

In [19]:
m = Message(name="ADT_A01")
m.pid.pid_5.pid_5_1 = last_name
m.pid.pid_5.pid_5_2 = first_name

In [20]:
print(m.to_er7())

PID|||||Mustermann^Max10321|||||2.5
